In [1]:
pip install findspark

Note: you may need to restart the kernel to use updated packages.


In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

# Criar a SparkSession
spark = SparkSession.builder \
    .appName("Agregações com PySpark") \
    .getOrCreate()

In [4]:
df_video = spark.read.parquet("C:/Users/Usuario/Documents/rede-social-yt/videos-preparados.snappy.parquet")



In [6]:
df_video.show()


+--------------------+-----------+------------+----------------+------+--------+---------+-----------+----+-----+-------------+--------------------+--------------------+--------------------+
|               Title|   Video ID|Published At|         Keyword| Likes|Comments|    Views|Interaction|Year|Month|Keyword Index|        Features PCA|     Features Normal|            Features|
+--------------------+-----------+------------+----------------+------+--------+---------+-----------+----+-----+-------------+--------------------+--------------------+--------------------+
|ASMR MUKBANG DOUB...|--ZI0dSbbNU|  2020-04-18|         mukbang|378858|   18860| 17975269|   18372987|2020|    4|         30.0|[0.6985786560867407]|[0.02303716158264...|[378858.0,1.79752...|
|Deadly car bomb d...|--hxd1CrOqg|  2022-08-22|            news|  6379|    4853|   808787|     820019|2022|    8|         37.0|[0.8936407990235931]|[3.87946679100418...|[6379.0,808787.0,...|
|How Biden&#39;s s...|--ixiTypG8g|  2022-08-2

In [7]:
df_keyword_count = df_video.groupBy("Keyword").count()
df_keyword_count.show()

+----------------+-----+
|         Keyword|count|
+----------------+-----+
|computer science|   48|
|            lofi|   40|
|         finance|   39|
|             cnn|   50|
|           apple|   42|
|            news|   39|
|         mukbang|   45|
|       education|   24|
|       interview|   50|
|          crypto|   50|
|   mathchemistry|   15|
|            food|   48|
|    data science|   50|
|        trolling|   50|
|        tutorial|   50|
|      literature|   46|
|             sat|   49|
|         history|   49|
|           cubes|   49|
|           music|   46|
+----------------+-----+
only showing top 20 rows



In [8]:
df_keyword_mean_interaction = df_video.groupBy("Keyword").avg("Interaction")
df_keyword_mean_interaction.show()


+----------------+--------------------+
|         Keyword|    avg(Interaction)|
+----------------+--------------------+
|computer science|  1226793.0208333333|
|            lofi|         4167085.875|
|         finance|   708542.9487179487|
|             cnn|           570650.86|
|           apple|1.0873628214285715E7|
|            news|  251688.71794871794|
|         mukbang|1.1053630377777778E7|
|       education|         2750838.625|
|       interview|          3044867.04|
|          crypto|            413676.2|
|   mathchemistry|  3427342.7333333334|
|            food|   5352944.104166667|
|    data science|           562465.28|
|        trolling|          1484584.88|
|        tutorial|           6936688.3|
|      literature|            881726.5|
|             sat|           1098927.0|
|         history| 1.565269257142857E7|
|           cubes|1.5043961224489795E7|
|           music|2.9691370304347824E7|
+----------------+--------------------+
only showing top 20 rows



In [9]:
df_keyword_max_interaction = df_video.groupBy("Keyword") \
    .max("Interaction") \
    .withColumnRenamed("max(Interaction)", "Rank Interactions") \
    .orderBy("Rank Interactions", ascending=False)
df_keyword_max_interaction.show()

+--------+-----------------+
| Keyword|Rank Interactions|
+--------+-----------------+
| animals|       1593623628|
|   music|        922551152|
|     bed|        532691631|
| history|        440187490|
|   apple|        429916936|
| mrbeast|        300397699|
|  google|        239385460|
|business|        210025196|
|   cubes|        170925917|
|  sports|        106924567|
| mukbang|         87433858|
|    lofi|         86445177|
|tutorial|         69616442|
|  movies|         65253870|
|  marvel|         56247330|
|  how-to|         53053975|
|    food|         48754479|
| physics|         43463298|
|    asmr|         34411125|
|nintendo|         32268486|
+--------+-----------------+
only showing top 20 rows



In [10]:
df_keyword_views_stats = df_video.groupBy("Keyword") \
    .agg(
        {"Views": "avg", "Views": "variance"}
    )
df_keyword_views_stats.show()

+----------------+--------------------+
|         Keyword|     variance(Views)|
+----------------+--------------------+
|computer science| 2.81219868165102E12|
|            lofi|1.846209641476677...|
|         finance|3.304483175097042...|
|             cnn|1.563423618468118...|
|           apple|4.299927977442589E15|
|            news|1.067512576672564...|
|         mukbang|5.586073238973179...|
|       education|1.833572249339214...|
|       interview|1.819220996034335E13|
|          crypto|3.513691634369074E12|
|   mathchemistry|2.491467065256849...|
|            food|7.326374128154842E13|
|    data science|5.479336525349994...|
|        trolling|6.932651793973286E12|
|        tutorial|1.369626596864457...|
|      literature|9.380521884205859E11|
|             sat|8.285094966049208E12|
|         history|4.253204661918686E15|
|           cubes|8.511756571903261E14|
|           music|1.924797107187940...|
+----------------+--------------------+
only showing top 20 rows



In [11]:
from pyspark.sql.functions import avg, min, max, round

df_keyword_views_summary = df_video.groupBy("Keyword") \
    .agg(
        round(avg("Views"), 0).alias("Avg Views"),
        round(min("Views"), 0).alias("Min Views"),
        round(max("Views"), 0).alias("Max Views")
    )
df_keyword_views_summary.show()

+----------------+-----------+---------+---------+
|         Keyword|  Avg Views|Min Views|Max Views|
+----------------+-----------+---------+---------+
|computer science|  1191959.0|    16115|  7004107|
|            lofi|  4089363.0|     6817| 84747957|
|         finance|   694223.0|     1195|  9450554|
|             cnn|   554240.0|    51269|  1889320|
|           apple| 1.074693E7|     1954|425478119|
|            news|   247492.0|    10642|  1465011|
|         mukbang|1.0904772E7|     3618| 86169225|
|       education|  2684433.0|     6611| 17103736|
|       interview|  2966112.0|     2587| 22529756|
|          crypto|   404608.0|     1599| 11805668|
|   mathchemistry|  3328125.0|       25| 18496859|
|            food|  5252406.0|    47430| 48018833|
|    data science|   544772.0|      911|  3069097|
|        trolling|  1420141.0|     5388| 14286302|
|        tutorial|  6761032.0|    19323| 68512549|
|      literature|   863021.0|     2847|  4231789|
|             sat|  1065869.0| 

In [12]:
from pyspark.sql.functions import first, last

df_keyword_published = df_video.groupBy("Keyword") \
    .agg(
        first("Published At").alias("First Published At"),
        last("Published At").alias("Last Published At")
    )
df_keyword_published.show()

+----------------+------------------+-----------------+
|         Keyword|First Published At|Last Published At|
+----------------+------------------+-----------------+
|computer science|        2022-02-08|       2020-09-08|
|            lofi|        2022-06-07|       2020-07-19|
|         finance|        2020-09-23|       2017-12-31|
|             cnn|        2022-08-17|       2022-08-13|
|           apple|        2022-08-22|       2022-08-02|
|            news|        2022-08-22|       2022-08-23|
|         mukbang|        2020-04-18|       2022-08-24|
|       education|        2015-02-06|       2010-10-14|
|       interview|        2021-08-03|       2018-10-05|
|          crypto|        2022-08-23|       2022-08-22|
|   mathchemistry|        2020-08-11|       2019-10-04|
|            food|        2022-07-17|       2022-08-20|
|    data science|        2019-08-18|       2021-08-06|
|        trolling|        2022-08-23|       2022-07-20|
|        tutorial|        2018-06-01|       2022

In [13]:
total_titles = df_video.count()
unique_titles = df_video.select("title").distinct().count()
print(f"Total Titles: {total_titles}")
print(f"Unique Titles: {unique_titles}")
print(f"Difference: {total_titles - unique_titles}")


Total Titles: 1869
Unique Titles: 1854
Difference: 15


In [14]:
from pyspark.sql.functions import year

df_video_year = df_video.withColumn("Year", year("Published At"))
df_year_count = df_video_year.groupBy("Year").count().orderBy("Year", ascending=True)
df_year_count.show()


+----+-----+
|Year|count|
+----+-----+
|2007|    2|
|2008|    1|
|2009|    9|
|2010|    6|
|2011|    4|
|2012|   12|
|2013|    6|
|2014|   10|
|2015|   15|
|2016|   34|
|2017|   47|
|2018|   57|
|2019|   86|
|2020|  158|
|2021|  229|
|2022| 1193|
+----+-----+



In [15]:
from pyspark.sql.functions import month

df_video_year_month = df_video.withColumn("Year", year("Published At")) \
                              .withColumn("Month", month("Published At"))
df_year_month_count = df_video_year_month.groupBy("Year", "Month").count().orderBy("Year", "Month", ascending=True)
df_year_month_count.show()


+----+-----+-----+
|Year|Month|count|
+----+-----+-----+
|2007|    7|    1|
|2007|   12|    1|
|2008|    7|    1|
|2009|    2|    2|
|2009|    6|    2|
|2009|    7|    1|
|2009|    8|    1|
|2009|   10|    1|
|2009|   12|    2|
|2010|    3|    1|
|2010|    5|    2|
|2010|    6|    1|
|2010|    9|    1|
|2010|   10|    1|
|2011|    2|    1|
|2011|    5|    1|
|2011|    9|    1|
|2011|   10|    1|
|2012|    1|    1|
|2012|    2|    3|
+----+-----+-----+
only showing top 20 rows



In [16]:
from pyspark.sql.window import Window
from pyspark.sql.functions import avg, sum as _sum

window_spec = Window.partitionBy("Keyword").orderBy("Year").rowsBetween(Window.unboundedPreceding, 0)

df_likes_cumulative = df_video_year \
    .groupBy("Keyword", "Year") \
    .agg(_sum("Likes").alias("Total Likes")) \
    .withColumn("Cumulative Likes", _sum("Total Likes").over(window_spec))
df_likes_cumulative.show()

+-------+----+-----------+----------------+
|Keyword|Year|Total Likes|Cumulative Likes|
+-------+----+-----------+----------------+
|animals|2009|    1357197|         1357197|
|animals|2010|     406734|         1763931|
|animals|2013|   11025176|        12789107|
|animals|2014|    6763260|        19552367|
|animals|2019|    1103713|        20656080|
|animals|2020|    6926869|        27582949|
|animals|2021|     901838|        28484787|
|animals|2022|     424693|        28909480|
|  apple|2016|    4144389|         4144389|
|  apple|2021|      38261|         4182650|
|  apple|2022|     776664|         4959314|
|   asmr|2020|     148120|          148120|
|   asmr|2021|    1089373|         1237493|
|   asmr|2022|     592709|         1830202|
|    bed|2007|     634321|          634321|
|    bed|2009|    2663647|         3297968|
|    bed|2010|     144517|         3442485|
|    bed|2011|    1532828|         4975313|
|    bed|2017|    1775692|         6751005|
|    bed|2018|    1900985|      